<a href="https://colab.research.google.com/github/ShuqairABD/Testing_ResNet_152_Dicom/blob/main/Abdalla_Shuqair/%D0%9C%D0%9E%D0%99%D0%95_Testing_ResNet_152_Dicom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Start**

In [ ]:
!pip install torch torchvision


In [ ]:
import torch
from torch.utils.data import Dataset, Subset, DataLoader
from torchvision import models, transforms
from torchvision.io import read_image, ImageReadMode
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import albumentations as A
from torchvision.transforms import v2
import numpy as np
import cv2
from PIL import Image
import PIL
from pathlib import Path
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy
from torchsummary import summary
from datetime import datetime
from tqdm import tqdm
import gc
import random
import pandas as pd
import pprint
from tabulate import tabulate

import torchvision.models as models
from torch.optim import Adam
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset


from torchvision import models, transforms, datasets



In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.3 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import pydicom
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **testing .dcm**

In [ ]:
!pip install pydicom


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.4 MB/s eta 0:00:00


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet152(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 1),
    nn.Sigmoid()
).to(device)


model_path = '/content/drive/MyDrive/resnet152_epoch_10.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((220, 220)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

label_mapping = {'Healthy': 0, 'Unhealthy': 1}

folder_path = '/content/drive/MyDrive/test_dcm'

def test_model(model, transform, label_mapping, folder_path, device):
    model.eval()

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.dcm'):

            image_path = os.path.join(folder_path, filename)
            dicom_image = pydicom.dcmread(image_path)
            image_array = dicom_image.pixel_array
            image = Image.fromarray(image_array).convert('RGB')
            input_image = transform(image).unsqueeze(0).to(device)


            with torch.no_grad():
                output = model(input_image)


            predicted = (output > 0.5).float()

            #class NAME
            predicted_label = predicted.item()
            predicted_class_name = list(label_mapping.keys())[list(label_mapping.values()).index(int(predicted_label))]

            print(f"Predicted class name for {filename}: {predicted_class_name}")

            plt.imshow(image)
            plt.title(f"Predicted class: {predicted_class_name}")
            plt.axis('off')
            plt.show()


test_model(model, transform, label_mapping, folder_path, device)


# **Copy weights to drive**

In [ ]:
!cp /content/fine_tuned4_resnet152.pth /content/drive/MyDrive/fine_tuned4_resnet152.pth
